In [ ]:
# this notebook demonstrate how to cuate dataset with the utitlity function provided
# it involved two steps, first we create or train appropriate agent, second is to sample those agents interaction with the environment and store them in a dataset

# Let's get the stock ticker we would use for curating the dataset

# this library is to get the list of tickers from NASDAQ and DOWJONES
from gettickers import *
nasdaq_tic = get_nasdaq_tickers()
dow_tic = get_dow_tickers()

# merge the lists and remove duplicates
all_tickers = nasdaq_tic + dow_tic
all_tickers = list(set(all_tickers))

# let's get the first 10 tickers and check the number of tickers in the lists
print(all_tickers[:10])
print(len(nasdaq_tic))
print(len(dow_tic))
print(len(all_tickers))

# agents can be created from RL (stable-baselines3)
# set date range for stock data to train RL-agents

# this will set  the parameter for the training of the agents
start_date = "2017-01-01"
num_days = 400
interval = "1d"
indicators = ["Volume", "volume_cmf", "trend_macd", "momentum_rsi", "momentum_stoch_rsi", "trend_sma_fast"]
init_balance = 20000
agent_output_path = "trained_stable_agents/"
stable_training_json_path = "training_stable_agents_config/"

# save these parameter as json config file using the utility function from getsttickers
create_json_files(all_tickers, start_date, num_days, interval, indicators, init_balance, agent_output_path, stable_training_json_path)

In [ ]:
# read one of the json file
import os
import json
json_file_path = os.listdir(stable_training_json_path)[0]
with open(os.path.join(stable_training_json_path, json_file_path), "r") as f:
    config = json.load(f)

print(config)

# extract the configuration parameters
stock_name = config['stock_name']
start_date = config['start_date']
num_days = config['num_days']
interval = config['interval']
indicators = config['indicators']
init_balance = config['init_balance']
output_path = config['output_path']

In [ ]:
# there are utility functions in curatedataset to help make gymnasium environment from the stock data
from curatedataset import makegymenv
stable_env, obs_space, act_space, col, data = makegymenv(stock_name=stock_name, start_date=start_date, period=num_days, interval=interval, indicators=indicators, normalize=False, init_balance=init_balance)

# Stable-baselines3 library can be used to create and train the agents
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

num_cpu = 6
# vectorized environment
env_vec = SubprocVecEnv([lambda: stable_env for i in range(num_cpu)])
# create the agent
modelA2C = A2C("MlpPolicy", env_vec, verbose=1)
# we can evaluate the agent before training
mean_reward, std_reward = evaluate_policy(modelA2C, Monitor(env_vec, n_eval_episodes=10, deterministic=False))
print(f"Pre-training -> mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

# train the agent
modelA2C.learn(total_timesteps=len(data)*80)

# evaluate the agent after training
mean_reward, std_reward = evaluate_policy(modelA2C, Monitor(env_vec, n_eval_episodes=10, deterministic=False))
print(f"Post training -> mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# the steps above can be used to train multiple agents and store them in a folder with the utility function in get_agent.py
from train_stable_agent import full_run

json_files = os.listdir(stable_training_json_path)

# loop through all json file and train stable-baseline agent with utility function
for file in json_files:
    # check if file ends with json
    if file.endswith(".json"):
        runfile = os.path.join(stable_training_json_path, file)
        print(f"Training agent with {runfile}")
        with open(runfile, "r") as f:
            runconfig = json.load(f)
        output_path = runconfig["output_path"]
        if os.path.exists(output_path) and len(os.listdir(output_path)) == 3:
            print(f"Agent with {runfile} already trained")
            continue
        else:
            try:
                full_run(runfile)
            except Exception as e:
                print(f"Error training agent with {runfile}")
                print(e)
                continue

In [ ]:
# now that stable-baseline agent is trained, we can go and sample them in the stock trading environment with different dates

from datetime import datetime, timedelta
# calculate the new start_date as the start_date + num_days for training stable-baseline agent
start_date = datetime.strptime(start_date, "%Y-%m-%d")
new_start_date = start_date + timedelta(days=num_days)

# we want to sample the agents for the days different between start date and now (today) minus 120 days (those 120 days for testing)
today = datetime.today().strftime("%Y-%m-%d")
today = datetime.strptime(today, "%Y-%m-%d")
day_diff = (today - new_start_date).days

if day_diff < 120:
    num_days = day_diff
else:
    num_days = day_diff - 120

new_start_date = new_start_date.strftime("%Y-%m-%d")
data_output_path = "offline_stock_trade_data/"
data_json_path = "offline_stock_trade_data_config/"

# save these parameter as json config file using the utility function from getsttickers
create_json_files(all_tickers, new_start_date, num_days, interval, indicators, init_balance, data_output_path, data_json_path)

In [ ]:
# read one of the json file
json_file_path = os.listdir(data_json_path)[0]

with open(os.path.join(data_json_path, json_file_path), "r") as f:
    config = json.load(f)

print(config)

# extract the configuration parameters
stock_name = config['stock_name']
start_date = config['start_date']
num_days = config['num_days']
interval = config['interval']
indicators = config['indicators']
init_balance = config['init_balance']
output_path = config['output_path']

In [ ]:
# make the gymnasium environment
data_env, obs_space, act_space, col, data = makegymenv(stock_name=stock_name, start_date=start_date, period=num_days, interval=interval, indicators=indicators, normalize=False, init_balance=init_balance)
# get the list of date from the data
env_date = data.index.strftime("%Y-%m-%d").tolist()
# get the folder where the trained agents are stored
agent_folder = os.path.join(agent_output_path, stock_name)
# get the a2c agent path
for file in os.listdir(agent_folder):
    if file.endswith(".zip") and "a2c" in file:
        a2c_agent_path = os.path.join(agent_folder, file)
        break
agent_type = (data_env, "stable-baselines-a2c", a2c_agent_path)